In [1]:
import os
import cv2
main_dir = "/kaggle/input/dataaseet/The IQ-OTHNCCD lung cancer dataset"
class_names = os.listdir(main_dir)

X_train=[]
y_train=[]

for class_name in class_names:
    class_dir = os.path.join(main_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, (299, 299))
            X_train.append(image)
            y_train.append(class_name)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split( X_train, y_train, test_size=0.3,  stratify=y_train ,random_state=42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,  # Random rotation between -10 and 10 degrees
    width_shift_range=0.1,  # Random horizontal shift of up to 10% of the width
    height_shift_range=0.1,  # Random vertical shift of up to 10% of the height
    shear_range=0.1,  # Random shearing transformation with a maximum shear of 10%
    zoom_range=0.1,  # Random zoom between 90% and 110%
    horizontal_flip=True,  # Random horizontal flip
    vertical_flip=True  # Random vertical flip
)

# Augment each image 5 times
augmented_images = []
augmented_labels = []
for image, label in zip((X_train), (y_train)):
    i = 0
    for batch in datagen.flow(tf.expand_dims(image, axis=0), batch_size=10):
        augmented_image = tf.squeeze(batch, axis=0)
        augmented_images.append(augmented_image)
        augmented_labels.append(label)
        i += 1
        if i >= 3:  # stop after augmenting 5 copies
            break

# Convert the augmented images to a numpy array
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

In [4]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

y_train = encoder.fit_transform(np.array(augmented_labels).reshape(-1, 1)).toarray()
#y_train = encoder.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()

y_test = encoder.transform(np.array(y_test).reshape(-1, 1)).toarray()
from keras.applications.xception import preprocess_input
X_train = preprocess_input(np.array(augmented_images))
X_test = preprocess_input(np.array(X_test))

In [5]:
import keras
from keras.layers import Conv2D, Activation, MaxPool2D, Flatten, Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Model

model=keras.applications.Xception(include_top=False, input_shape=(299, 299,3), weights='/kaggle/input/xcept-we/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')

    # Freeze all the layers
for layer in model.layers[:-20]:
    layer.trainable = False

output = model.output
#flatten layer

output= GlobalAveragePooling2D()(output)
output =Flatten()(output)
#Fully Connection Layers
# FC1

output =Dense(512, activation="relu")(output)
output =Dense(256, activation="relu")(output)
# FC7
output =Dense(64, activation="relu")(output)
# FC8
output =Dense(5,activation="softmax")(output)
model = Model(model.input, output)
CB = EarlyStopping(monitor=' val_loss', patience=3,restore_best_weights = True)

In [6]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['Accuracy'])
history = model.fit(X_train, y_train ,  epochs=15, batch_size=22 ,validation_data=(X_test,y_test),callbacks=[CB])

Epoch 1/15
177/177 [==============================] - 53s 234ms/step - loss: 0.4341 - Accuracy: 0.8371 - val_loss: 0.7891 - val_Accuracy: 0.7928
Epoch 2/15
177/177 [==============================] - 40s 228ms/step - loss: 0.1646 - Accuracy: 0.9412 - val_loss: 0.2321 - val_Accuracy: 0.9315
Epoch 3/15
177/177 [==============================] - 41s 230ms/step - loss: 0.1137 - Accuracy: 0.9626 - val_loss: 0.1942 - val_Accuracy: 0.9622
Epoch 4/15
177/177 [==============================] - 40s 229ms/step - loss: 0.0506 - Accuracy: 0.9817 - val_loss: 0.3101 - val_Accuracy: 0.9225
Epoch 5/15
177/177 [==============================] - 41s 230ms/step - loss: 0.0701 - Accuracy: 0.9804 - val_loss: 0.2089 - val_Accuracy: 0.9514
Epoch 6/15
177/177 [==============================] - 40s 229ms/step - loss: 0.0367 - Accuracy: 0.9912 - val_loss: 0.2627 - val_Accuracy: 0.9495
Epoch 7/15
177/177 [==============================] - 41s 230ms/step - loss: 0.0491 - Accuracy: 0.9884 - val_loss: 1.3355 - val_Ac

In [7]:
model.evaluate(X_test,y_test)

18/18 [==============================] - 8s 281ms/step - loss: 0.4931 - Accuracy: 0.9153


[0.49308720231056213, 0.9153153300285339]

In [8]:
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))

18/18 [==============================] - 5s 223ms/step
0.9153153153153153
                 precision    recall  f1-score   support

      pneumonia       1.00      0.19      0.33        36
   Normal cases       1.00      1.00      1.00       144
   Bengin cases       0.99      0.91      0.95       151
Malignant cases       0.81      0.97      0.88       125
         Corona       0.85      1.00      0.92        99

       accuracy                           0.92       555
      macro avg       0.93      0.81      0.81       555
   weighted avg       0.93      0.92      0.90       555

